In [2]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
load_dotenv()

URI = os.getenv("NEO4J_URI")
AUTH = os.getenv("NEO4J_PASSWORD")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

RuntimeError: CPU dispatcher tracer already initlized

In [4]:
!pip install -U ragas


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
import csv
questions = []
ground_truths = []
final_test_csv_path = '/Volumes/SandiskSSD/Downloads/combined_50_rows.csv'
with open(final_test_csv_path, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        questions.append(row.get('question', "").strip())
        ground_truths.append(row.get('ground_truth', "").strip())

print(questions[:5])
print(ground_truths[:5])

['A mother in a crowded urban area in a tropical country is worried about her 9-month-old infant. The infant has been experiencing a persistent cough, severe difficulty breathing, very fast breathing, noticeable chest indrawing, and has been struggling to feed for the past day. The infant is malnourished, but no other known pre-existing medical conditions are reported. The family uses biomass fuel for cooking in their home, but no other significant lifestyle risk factors are reported. What could be causing these severe respiratory symptoms in her child?', "I'm a 32-year-old woman in West Africa who recently gave birth to her third child two months ago. I've been experiencing massive swelling in my face and legs, difficulty breathing, constant fatigue, and sometimes feel lightheaded. My blood pressure readings at home have also been noticeably high. I have no known pre-existing heart conditions, though I experienced mild hypertension during my recent pregnancy, and I live in a hot, humi

In [81]:
len(ground_truths)

50

In [82]:
ground_truths

['Diagnosis: Severe acute bacterial pneumonia.\nExplanation: The infant is likely suffering from a life-threatening infection often caused by Streptococcus pneumoniae or Haemophilus influenzae. Symptoms like fast breathing and chest indrawing are hallmark signs of lower respiratory tract involvement in tropical settings. Malnutrition and exposure to indoor smoke from biomass fuel are critical risk factors that increase the severity of this condition. Immediate medical intervention is necessary due to the high mortality risk in vulnerable infants.',
 'Diagnosis: Peripartum Cardiac Failure (PPCF).\nExplanation: The symptoms described are highly consistent with PPCF, a syndrome typically manifesting in the final month of pregnancy or within five months postpartum. The presentation of massive oedema, difficulty breathing due to pulmonary congestion, and fatigue points to heart failure. In West Africa, traditional practices such as consuming high-sodium rock salt (kanwa) combined with a hot

In [83]:
len(questions)

50

In [84]:
questions

['A mother in a crowded urban area in a tropical country is worried about her 9-month-old infant. The infant has been experiencing a persistent cough, severe difficulty breathing, very fast breathing, noticeable chest indrawing, and has been struggling to feed for the past day. The infant is malnourished, but no other known pre-existing medical conditions are reported. The family uses biomass fuel for cooking in their home, but no other significant lifestyle risk factors are reported. What could be causing these severe respiratory symptoms in her child?',
 "I'm a 32-year-old woman in West Africa who recently gave birth to her third child two months ago. I've been experiencing massive swelling in my face and legs, difficulty breathing, constant fatigue, and sometimes feel lightheaded. My blood pressure readings at home have also been noticeably high. I have no known pre-existing heart conditions, though I experienced mild hypertension during my recent pregnancy, and I live in a hot, hum

In [6]:
from Generate_Response.trindsLangchain import with_naive_kg, with_naive_only, generate_response_with_context

RuntimeError: CPU dispatcher tracer already initlized

In [93]:
contexts = []
for question in questions:
    context = with_naive_kg(question)
    contexts.append(context)

In [94]:
len(contexts)

50

In [117]:
contexts

[["\nMAIN INFORMATION:\nSymptoms: Skin, bladder and lung cancers, hyperpigmentation, keratoses\nDiagnosis: Chronic Arsenic Poisoning (Arsenicism)\nEpidemiology: Arsenic levels in drinking water documented in several tropical countries such as India, Chile, Mexico, Bangladesh, Peru, Taiwan, Mongolia and Thailand, as well as in parts of the US. Millions exposed to arsenic-contaminated drinking water across Bangladesh and West Bengal in India.\nInvestigations: Diagnosis usually rests on direct measurement of blood lead levels. Basophilic stippling of erythrocytes and gingival lead lines may be seen.\nRisk Factors & Patient Profile: Exposure to arsenic occurs mainly through ingestion of contaminated water from deep wells, food, or inhalation of arsenic fumes. Arsenic is leached into drinking water supplies from bedrock.\n---\nADDITIONAL CONTEXTS:\n[INDOOR AIR POLLUTION]\nContent: The health effects associated with indoor air pollution (IAP) are of great concern in developing countries, whe

In [103]:
contexts_json = []
for context in contexts:
    context_json = {
        "contexts": context
    }
    contexts_json.append(context_json)


In [104]:
len(contexts_json)

50

## Save as json file so that next to we don't need to run generate_contexts() above again

In [105]:
import json
file_path = "/Volumes/SandiskSSD/Downloads/ragas_evaluation/contexts_qdrant_neo4j_50_no_reranking.json"

# 3. Open the file in write mode and dump the data
try:
    with open(file_path, 'w') as json_file:
        json.dump(contexts_json, json_file, indent=4)
    print(f"Successfully converted list to '{file_path}'")
except IOError as e:
    print(f"Error writing to file: {e}")

Successfully converted list to '/Volumes/SandiskSSD/Downloads/ragas_evaluation/contexts_qdrant_neo4j_50_no_reranking.json'


In [110]:
import json
def json_file_to_list(filename):
    """
    Converts the contents of a JSON file into a Python list.

    Args:
        filename (str): The path to the JSON file.

    Returns:
        list: The Python list representation of the JSON data.
    """
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
            if isinstance(data, list):
                return data
            else:
                print(f"Error: JSON data in '{filename}' is not a list at the root.")
                return []
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return []
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from the file '{filename}'.")
        return []

# Usage:
file_name = '/Volumes/SandiskSSD/Downloads/ragas_evaluation/contexts_qdrant_neo4j_50_no_reranking.json'
my_list = json_file_to_list(file_name)

final_contexts = []
for item in my_list:
    final_contexts.append(item.get("contexts"))


In [116]:
final_contexts

[["\nMAIN INFORMATION:\nSymptoms: Skin, bladder and lung cancers, hyperpigmentation, keratoses\nDiagnosis: Chronic Arsenic Poisoning (Arsenicism)\nEpidemiology: Arsenic levels in drinking water documented in several tropical countries such as India, Chile, Mexico, Bangladesh, Peru, Taiwan, Mongolia and Thailand, as well as in parts of the US. Millions exposed to arsenic-contaminated drinking water across Bangladesh and West Bengal in India.\nInvestigations: Diagnosis usually rests on direct measurement of blood lead levels. Basophilic stippling of erythrocytes and gingival lead lines may be seen.\nRisk Factors & Patient Profile: Exposure to arsenic occurs mainly through ingestion of contaminated water from deep wells, food, or inhalation of arsenic fumes. Arsenic is leached into drinking water supplies from bedrock.\n---\nADDITIONAL CONTEXTS:\n[INDOOR AIR POLLUTION]\nContent: The health effects associated with indoor air pollution (IAP) are of great concern in developing countries, whe

In [115]:
len(contexts)

50

In [118]:
len(questions)

50

In [ ]:
import asyncio

answers_list = []

for question, context in zip(questions, contexts):
    # No index needed!
    answers_list.append(generate_response_with_context(question, context))

# Run all tasks at the same time
answers = await asyncio.gather(*answers_list)

In [51]:
len(answers)

50

In [82]:
answers_json = []
for answer in answers:
    answer_json = {
        "answer": answer
    }
    answers_json.append(answer_json)


In [86]:
import json
file_path = "/Volumes/SandiskSSD/Downloads/answers_qwen25_qdrant_neo4j.json"

# 3. Open the file in write mode and dump the data
try:
    with open(file_path, 'w') as json_file:
        json.dump(answers_json, json_file, indent=4)
    print(f"Successfully converted list to '{file_path}'")
except IOError as e:
    print(f"Error writing to file: {e}")

Successfully converted list to '/Volumes/SandiskSSD/Downloads/answers_qwen25_qdrant_neo4j.json'


In [53]:
answers

['Diagnosis: Infantile beriberi\nExplanation: This presentation is highly characteristic of infantile beriberi caused by thiamine deficiency. The key diagnostic clues include the classic clinical features described, such as dyspnoea (difficulty breathing), cyanosis (blue or purple skin due to lack of oxygen in the blood), irritability, grunting while breathing, poor breastfeeding ability ("poor suck"), an abnormal cry that may be weak and high-pitched ("high-pitched cry" is not explicitly mentioned but implied by "abnormal cry"), swelling of the extremities ("swollen limbs"), a history consistent with thiamine deficiency (specifically mentioning prolonged postpartum food avoidance), and confirmation through blood analysis showing low levels of thiamine.',
 "Diagnosis: Pregnancy-Induced Hypertension (Preeclampsia)\nExplanation: The user's presentation is highly characteristic of Preeclampsia, an acute hypertensive disorder specific to pregnancy and the postpartum period. It typically ma

In [486]:
contexts

[["\nMAIN INFORMATION:\nSymptoms: Skin, bladder and lung cancers, hyperpigmentation, keratoses\nDiagnosis: Chronic Arsenic Poisoning (Arsenicism)\nEpidemiology: Arsenic levels in drinking water documented in several tropical countries such as India, Chile, Mexico, Bangladesh, Peru, Taiwan, Mongolia and Thailand, as well as in parts of the US. Millions exposed to arsenic-contaminated drinking water across Bangladesh and West Bengal in India.\nInvestigations: Diagnosis usually rests on direct measurement of blood lead levels. Basophilic stippling of erythrocytes and gingival lead lines may be seen.\nRisk Factors & Patient Profile: Exposure to arsenic occurs mainly through ingestion of contaminated water from deep wells, food, or inhalation of arsenic fumes. Arsenic is leached into drinking water supplies from bedrock.\n---\nADDITIONAL CONTEXTS:\n[INDOOR AIR POLLUTION]\nContent: The health effects associated with indoor air pollution (IAP) are of great concern in developing countries, whe

In [56]:
len(ground_truths)

50

In [57]:
len(questions)

50

In [58]:
from datasets import Dataset
import os
from ragas import evaluate
from langchain_openai import ChatOpenAI
from ragas.metrics import faithfulness, answer_correctness, answer_relevancy, answer_similarity, context_precision, context_recall, _factual_correctness
from dotenv import load_dotenv

load_dotenv()

eval_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0, model_kwargs={"response_format": {"type": "json_object"}})

metrics_list = [
    faithfulness,
    answer_correctness,
    answer_relevancy,
    answer_similarity,
    context_precision,
    context_recall,
]


for metric in metrics_list:
    metric.llm = eval_llm


data_samples = {
    'question': questions,
    'answer': answers,
    'contexts': contexts,
    'ground_truth': ground_truths
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=metrics_list)
df = score.to_pandas()
df.to_csv('ragas_qwen25_ft_rag.csv', index=False)

/var/folders/qh/10rj2tgd6dld_075lmzprz_h0000gn/T/ipykernel_60379/1828086049.py:5: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_correctness, answer_relevancy, answer_similarity, _context_precision, _context_recall, _factual_correctness
/var/folders/qh/10rj2tgd6dld_075lmzprz_h0000gn/T/ipykernel_60379/1828086049.py:5: DeprecationWarning: Importing answer_correctness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_correctness
  from ragas.metrics import faithfulness, answer_correctness, answer_relevancy, answer_similarity, _context_precision, _context_recall, _factual_correctness
/var/folders/qh/10rj2tgd6dld_075lmzprz_h0000gn/T/ipykernel_60379/1828086049.py:5

Evaluating:   0%|          | 0/200 [00:00<?, ?it/s]

In [60]:
import pandas as pd

# 1. Read the CSV files into pandas DataFrames
df1 = pd.read_csv('ragas_qwen25_ft_rag.csv')

average_faithfulness = df1['faithfulness'].mean()
average_answer_correctness = df1['answer_correctness'].mean()
average_answer_relevancy = df1['answer_relevancy'].mean()
average_answer_similarity = df1['answer_similarity'].mean()
average_context_precision = df1['context_precision'].mean()
average_context_recall = df1['context_recall'].mean()

# 3. Save the result to a new CSV file
df1.to_csv('ragas_qwen25_ft_rag.csv', index=False)


In [384]:
print(f"Average Faithfulness: {average_faithfulness}")
print(f"Average Context Precision: {average_context_precision}")
print(f"Average Context Recall: {average_context_recall}")

Average Faithfulness: 0.20049411699411704
Average Context Precision: 0.5066666666258333
Average Context Recall: 0.25433333333333336


In [386]:
print(f"Average Faithfulness: {average_faithfulness}")
print(f"Average Context Precision: {average_context_precision}")
print(f"Average Context Recall: {average_context_recall}")

Average Faithfulness: 0.2642128427128427
Average Context Precision: 0.4266666666338334
Average Context Recall: 0.26066666666666666


In [443]:
print(f"Average Faithfulness: {average_faithfulness}")
print(f"Average Context Precision: {average_context_precision}")
print(f"Average Context Recall: {average_context_recall}")

Average Faithfulness: 0.2736535686535686
Average Context Precision: 0.5766666666338334
Average Context Recall: 0.32866666666666666


In [468]:
print(f"Average Faithfulness: {average_faithfulness}")
print(f"Average Context Precision: {average_context_precision}")
print(f"Average Context Recall: {average_context_recall}")

Average Faithfulness: 0.2971471306471306
Average Context Precision: 0.5833333332926667
Average Context Recall: 0.30866666666666664


In [507]:
print(f"Average Faithfulness: {average_faithfulness}")
print(f"Average Answer Correctness: {average_answer_correctness}")
print(f"Average Answer Relevancy: {average_answer_relevancy}")
print(f"Average Answer Similarity: {average_answer_similarity}")

Average Faithfulness: 0.3912518191532123
Average Answer Correctness: 0.45817676969944465
Average Answer Relevancy: 0.8189513124198672
Average Answer Similarity: 0.901324345229239


In [61]:
print(f"Average Faithfulness: {average_faithfulness}")
print(f"Average Answer Correctness: {average_answer_correctness}")
print(f"Average Answer Relevancy: {average_answer_relevancy}")
print(f"Average Answer Similarity: {average_answer_similarity}")

Average Faithfulness: 0.4260697981756805
Average Answer Correctness: 0.4192301699893802
Average Answer Relevancy: 0.8062596596241605
Average Answer Similarity: 0.8948068405020977


In [381]:
combined_df

,user_input,retrieved_contexts,response,reference,faithfulness,context_precision,context_recall
0,A mother in a crowded urban area in a tropical...,"[""[EXPOSURE AND EPIDEMIOLOGY]\nDisease: Infant...",Diagnosis: Pneumonia\nExplanation: The infant'...,Diagnosis: Severe acute bacterial pneumonia.\n...,0.100000,0.333333,0.000000
1,I'm a 32-year-old woman in West Africa who rec...,['[CLINICAL FEATURES]\nContent: In severe case...,Diagnosis: Postpartum hypertension with possib...,Diagnosis: Peripartum Cardiac Failure (PPCF).\...,0.000000,1.000000,0.400000
2,A 35-year-old woman in a sub-Saharan African c...,"[""[PATIENT INFORMATION]\nDisease: Strongyloide...",Diagnosis: HIV-associated tuberculous pericard...,Diagnosis: Mycobacterium tuberculosis pericard...,0.714286,0.500000,0.750000
3,"A 45-year-old woman in a local hospital, with ...",['[OUTCOMES OF BLOOD TRANSFUSIONS]\nContent: I...,Diagnosis: Acute Hemolytic Transfusion Reactio...,Diagnosis: Acute Hemolytic Transfusion Reactio...,0.000000,0.333333,0.000000
4,"I'm a 30-year-old man in Southeast Asia, with ...","[""[HISTORY OF PRESENT ILLNESS]\nDisease: Rabie...",Diagnosis: Dengue fever with neurological invo...,Diagnosis: Acute viral encephalitis.\nExplanat...,0.000000,0.333333,0.400000
5,A 35-year-old male living in the Mediterranean...,"[""[HISTORY OF PRESENT ILLNESS]\nDisease: Afric...",Diagnosis: Rocky Mountain Spotted Fever \nExp...,Diagnosis: Rickettsial infection (likely Medit...,0.285714,1.000000,0.600000
6,"My friend, a 36-year-old man, recently returne...","[""[HISTORY OF PRESENT ILLNESS]\nDisease: Afric...",Diagnosis: Cerebral malaria\nExplanation: The ...,Diagnosis: Cerebral malaria.\nExplanation: The...,0.111111,0.500000,0.200000
7,I'm a 45-year-old woman living in Southeast As...,"['[NEUROLOGICAL SYMPTOMS]\nContent: Rarely, pa...",Diagnosis: Eosinophilic meningitis due to Angi...,Diagnosis: Cerebral paragonimiasis.\nExplanati...,1.000000,1.000000,0.000000
8,I'm a 44-year-old woman living in a developing...,['[PATIENT INFORMATION]\nDisease: Scrub typhus...,Diagnosis: Trachoma \nExplanation: The patien...,Diagnosis: Trachoma.\nExplanation: The patient...,0.400000,0.500000,1.000000
9,"I'm a 30-year-old woman in Southeast Asia, and...","[""[HISTORY OF PRESENT ILLNESS]\nDisease: Histo...",Diagnosis: Cutaneous leishmaniasis \nExplanat...,Diagnosis: Yaws. \nExplanation: The described ...,0.100000,0.333333,0.000000


In [304]:
from ragas.metrics.collections import ContextPrecision
from ragas.metrics._context_precision import ContextPrecision
from datasets import Dataset

# 1. Correct Imports
from ragas import evaluate

async def main():
    data = {
        'question': [
            "Where is the Eiffel Tower located?",
            "What is the capital of Japan?"
        ],
        'ground_truth': [
            "The Eiffel Tower is located in Paris.",
            "Tokyo is the capital of Japan."
        ],
        'contexts': [
            ["The Eiffel Tower is located in Paris.", "The Brandenburg Gate is in Berlin."],
            ["Kyoto was the old capital.", "Tokyo is the current capital of Japan."]
        ]
    }

    # 2. Create Dataset
    dataset = Dataset.from_dict(data)

    # 3. Setup LLM
    # Ensure OPENAI_API_KEY is set in environment
    client = AsyncOpenAI()
    llm = llm_factory("gpt-4o-mini", client=client)

    # 4. Initialize Metric (Now valid)
    context_precision = ContextPrecision(llm=llm)

    # 5. Run Evaluation
    results = evaluate(
        dataset=dataset,
        metrics=[context_precision],
        llm=llm
    )

    print(results)
    df = results.to_pandas()
    print(df)

# Run in notebook
await main()

/var/folders/qh/10rj2tgd6dld_075lmzprz_h0000gn/T/ipykernel_43494/3058685482.py:5: DeprecationWarning: Importing ContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import ContextPrecision
  from ragas.metrics import ContextPrecision  # <--- FIXED IMPORT


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

{'context_precision': 0.7500}
                           user_input  \
0  Where is the Eiffel Tower located?   
1       What is the capital of Japan?   

                                  retrieved_contexts  \
0  [The Eiffel Tower is located in Paris., The Br...   
1  [Kyoto was the old capital., Tokyo is the curr...   

                               reference  context_precision  
0  The Eiffel Tower is located in Paris.                1.0  
1         Tokyo is the capital of Japan.                0.5  


In [38]:
from openai import AsyncOpenAI
from ragas.llms import llm_factory
from ragas.metrics.collections import NoiseSensitivity

# Setup LLM
client = AsyncOpenAI()
llm = llm_factory("gpt-4o-mini", client=client)

# Create metric
scorer = NoiseSensitivity(llm=llm)

# Evaluate
result = await scorer.ascore(
    user_input="What is the Life Insurance Corporation of India (LIC) known for?",
    response="The Life Insurance Corporation of India (LIC) is the largest insurance company in India, known for its vast portfolio of investments. LIC contributes to the financial stability of the country.",
    reference="The Life Insurance Corporation of India (LIC) is the largest insurance company in India, established in 1956 through the nationalization of the insurance industry. It is known for managing a large portfolio of investments.",
    retrieved_contexts=[
        "The Life Insurance Corporation of India (LIC) was established in 1956 following the nationalization of the insurance industry in India.",
        "LIC is the largest insurance company in India, with a vast network of policyholders and huge investments.",
        "As the largest institutional investor in India, LIC manages substantial funds, contributing to the financial stability of the country.",
        "The Indian economy is one of the fastest-growing major economies in the world, thanks to sectors like finance, technology, manufacturing etc."
    ]
)
print(f"Noise Sensitivity Score: {result.value}")

Noise Sensitivity Score: 0.3333333333333333
